# BaseLine

### Import library

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import xgboost as xgb
import re
from datetime import datetime
import os

### Load Data

In [4]:
train = pd.read_csv('./data/open/train.csv')
test = pd.read_csv("./data/open/test.csv")

### Data Preprocessing

In [8]:
# 변이 유형 라벨링 함수
def label_mutation_type(mutation_code):
    if pd.isna(mutation_code):
        return 'Unknown'
    if isinstance(mutation_code, (int, float)):
        return 'Numeric'
    if not isinstance(mutation_code, str):
        return 'Unknown'
    if mutation_code == 'WT':
        return 'Wild Type'
    elif '*' in mutation_code:
        return 'Deletion'
    elif 'fs' in mutation_code:
        return 'Frameshift'
    else:
        return 'Missense'

# 변이 정보 처리 함수
def process_mutations(row):
    mutations = []
    for col in row.index:
        if col not in ['ID', 'SUBCLASS']:
            mutation_type = label_mutation_type(row[col])
            mutations.append(f"{col}:{mutation_type}")
    return ' '.join(mutations)

In [9]:
# 훈련 데이터와 테스트 데이터에 전처리 적용
train['mutations'] = train.apply(process_mutations, axis=1)
test['mutations'] = test.apply(process_mutations, axis=1)

# 특성 추출
def extract_features(df):
    # 각 변이 유형의 개수 계산
    df['wild_type_count'] = df['mutations'].str.count('Wild Type')
    df['deletion_count'] = df['mutations'].str.count('Deletion')
    df['frameshift_count'] = df['mutations'].str.count('Frameshift')
    df['missense_count'] = df['mutations'].str.count('Missense')
    df['unknown_count'] = df['mutations'].str.count('Unknown')
    df['numeric_count'] = df['mutations'].str.count('Numeric')
    
    # 전체 변이 개수
    df['total_mutations'] = df['mutations'].str.count(':') - df['wild_type_count'] - df['unknown_count'] - df['numeric_count']
    
    return df

train = extract_features(train)
test = extract_features(test)

# 특성과 타겟 분리
feature_columns = ['wild_type_count', 'deletion_count', 'frameshift_count', 'missense_count', 'unknown_count', 'numeric_count', 'total_mutations']
X_train = train[feature_columns]
y_train = train['SUBCLASS']
X_test = test[feature_columns]


In [10]:
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
# 타겟 변수 인코딩
le_subclass = LabelEncoder()
y_train = le_subclass.fit_transform(train['SUBCLASS'])

# 변환된 레이블 확인
print("변환된 레이블:")
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

변환된 레이블:
원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25


### Model define & Train

In [11]:
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss' 
)

In [12]:
model.fit(X_train, y_train)

c:\Users\wjdwn\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:31:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

### Inference

In [13]:
predictions = model.predict(X_test)

In [14]:
original_labels = le_subclass.inverse_transform(predictions)

### Submisson

In [19]:
# 날짜와 시간을 포함한 폴더 생성 함수
def create_timestamp_folder(base_path='./submissions'):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder_path = os.path.join(base_path, timestamp)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

In [22]:
submission = pd.read_csv("./data/open/sample_submission.csv")
submission["SUBCLASS"] = original_labels

# 타임스탬프 폴더 생성
submission_folder = create_timestamp_folder()

submission_path = os.path.join(submission_folder, 'submission.csv')
submission.to_csv(submission_path, encoding='UTF-8-sig', index=False)